# Power Flow with Complementarity Constraints

In [ ]:
import optalg
import pfnet

## Parse network

In [ ]:
parser = pfnet.ParserMAT()
net = parser.parse('../../cases/case3375wp.mat')

## Construct problem

In [ ]:
net.set_flags('bus','variable','not slack',['voltage magnitude','voltage angle'])
net.set_flags('generator','variable','slack','active power')
net.set_flags('generator','variable','regulator','reactive power')

problem = pfnet.Problem(net)
problem.add_constraint(pfnet.Constraint('AC power balance',net))
problem.add_constraint(pfnet.Constraint('generator active power participation',net))
problem.add_constraint(pfnet.Constraint('generator reactive power participation',net))
problem.add_constraint(pfnet.Constraint('voltage regulation by generators',net))
problem.add_function(pfnet.Function('voltage magnitude regularization',1.,net))
problem.analyze()
problem.show()

## Solve problem

In [ ]:
solver = optalg.opt_solver.OptSolverAugL()
solver.set_parameters({'quiet':True, 'kappa':1e-5})
solver.solve(problem)
print(solver.get_status(), solver.get_iterations())

## Get results

In [ ]:
net.update_properties()
print(net.bus_P_mis, net.bus_Q_mis)
net.set_var_values(solver.get_primal_variables())
net.update_properties()
print(net.bus_P_mis, net.bus_Q_mis)